In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r ../requirements.txt

import PETs_Experiment

In [2]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [3]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 1 # 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   27   Private  205145   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Outlierist (IQR): Dropped  1081 rows on fnlwgt         . Kept [-63379.5, 418648.5] only.
Preprocessor - Outlierist (IQR): Dropped   214 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3043 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   233 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1719 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6081 in 36179 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column occupation      been labelized from 0 to 13.
Preprocessor - Encoder (Lab

In [5]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0359 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.4888 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30098 rows (same as raw) in 1.3257 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   48          2 -1.606896         13               11               2   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           9             0     4       1             0             0   

   hours-per-week  native-country  income  
0              58              39       0  


In [6]:
synthesizer.data_syn.dtypes

age                   int8
workclass            int32
fnlwgt             float64
education            int32
educational-num       int8
marital-status       int32
occupation           int32
relationship         int32
race                 int32
gender               int32
capital-gain         int32
capital-loss         int16
hours-per-week        int8
native-country       int32
income               int32
dtype: object

In [7]:
preproc.missingist['education'].na_percentage

0.0

In [8]:
# File D:\Dropbox\89_其他應用\GitHub\PETs-Experiment\PETs_Experiment\Preprocessor\Missingist.py:27, in Missingist.inverse_transform(self, col_data)
#      25 _col_data = col_data.copy()
#      26 print(_col_data)
# ---> 27 _col_data[_na_mask] = np.nan
#      28 return _col_data

# ValueError: cannot convert float NaN to integer

postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        # ,missingist = getattr(preproc ,'missingist' ,None)
                                        ,encoder    = getattr(preproc ,'encoder'    ,None)
                                        ,scaler     = getattr(preproc ,'scaler'     ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
   age workclass        fnlwgt     education  educational-num  \
0   48   Private  38910.569792  Some-college               11   

       marital-status      occupation relationship   race gender  \
0  Married-civ-spouse  Prof-specialty      Husband  White   Male   

   capital-gain  capital-loss  hours-per-week native-country income  
0             0             0              58     Yugoslavia 

In [9]:
eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-singlingout-univariate'
                                 ,data = {'ori'      : splitter.data[1]['train']
                                          ,'syn'     : postproc.data
                                          ,'control' : splitter.data[1]['validation']
                                 }
                                 ,anonymeter_n_attacks = 2000
                                 )
eval.eval()
eval.Evaluator.evaluation

Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.6022 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 809 failed queries out of 2000. Check DEBUG messages for more details.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.0013568577126237004,
 'Attack_Rate': 0.0009585236406264672,
 'Attack_Rate_err': 0.0009585236406264671,
 'Baseline_Rate': 0.0009585236406264672,
 'Baseline_Rate_err': 0.0009585236406264671,
 'Control_Rate': 0.0009585236406264672,
 'Control_Rate_err': 0.0009585236406264671}

In [10]:
eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-linkability'
                                 ,data = {'ori'      : splitter.data[1]['train']
                                          ,'syn'     : postproc.data
                                          ,'control' : splitter.data[1]['validation']
                                 }
                                 ,anonymeter_n_attacks   = 2000
                                 ,anonymeter_n_neighbors = 10
                                 ,anonymeter_n_jobs      = -1
                                 ,anonymeter_aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
                                                        ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
                                                        ]
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (Anonymeter - Linkability): Now is Linkability Evaluator
Evaluator (Anonymeter - Linkability): aux_cols are [age, fnlwgt, race, gender, native-country]
Evaluator (Anonymeter - Linkability): and [workclass, education, capital-gain, capital-loss, hours-per-week].
Evaluator (Anonymeter - Linkability): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Linkability.
Evaluator (Anonymeter): Evaluating  Linkability spent 6.7687 sec.


{'Risk': 0.001498560835065448,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.00366871311019686,
 'Attack_Rate': 0.002455648069704588,
 'Attack_Rate_err': 0.0019453844860661056,
 'Baseline_Rate': 0.003453731022423335,
 'Baseline_Rate_err': 0.00238542229400942,
 'Control_Rate': 0.0009585236406264672,
 'Control_Rate_err': 0.0009585236406264671}

In [11]:
eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-inference'
                                 ,data = {'ori'      : splitter.data[1]['train']
                                          ,'syn'     : postproc.data
                                          ,'control' : splitter.data[1]['validation']
                                 }
                                 ,anonymeter_n_attacks   = 2000
                                 ,anonymeter_n_jobs      = -1
                                 ,anonymeter_secret = 'age'
                                 )
eval.eval()
eval.Evaluator.evaluation

Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 0.8181 sec.


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.02944448518616154,
 'Attack_Rate': 0.08320222299385194,
 'Attack_Rate_err': 0.023904093583999354,
 'Baseline_Rate': 0.08518697431292883,
 'Baseline_Rate_err': 0.024167884844459902,
 'Control_Rate': 0.09114122827015952,
 'Control_Rate_err': 0.0249352437952306}

In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment


para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
    ,'Evaluator_evaluating_method'    : 'anonymeter-singlingout-univariate' # 'anonymeter-inference' # 
    ,'Evaluator_anonymeter_n_attacks' : 1 # 2000
    ,'Evaluator_num_samples'          : 2
}

executor_single = PETs_Experiment.Executor(**para_Executor)
executor_single.run_single_process()
executor_single.evaluator[('[Adt Income] adult.csv',
  '0.8',
  1,
  'drop-IQR-Label-Standard',
  'sdv-singletable-gaussiancoupula',
  'anonymeter-singlingout-univariate',
  1)].Evaluator.evaluation

executor_single = PETs_Experiment.Executor(**para_Executor)
executor_single.run()
executor_single.evaluator[('[Adt Income] adult.csv',
  '0.8',
  1,
  'drop-IQR-Label-Standard',
  'sdv-singletable-gaussiancoupula',
  'anonymeter-singlingout-univariate',
  1)].Evaluator.evaluation

Executor - Loader: [Adt Income] adult.csv loading time: 11.1665 sec.
Executor - Splitter: 0.8 splitting time: 0.1006 sec.
Preprocessor - Outlierist (IQR): Dropped  1681 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1077 rows on fnlwgt         . Kept [-62094.5, 416317.5] only.
Preprocessor - Outlierist (IQR): Dropped   235 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   220 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3027 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6042 in 36201 rows.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column occupation      been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column native-country  been labelized fro

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 17.8809 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30159 rows (same as raw) in 1.5327 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 23.4787 sec.
Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Executor - Postprocessor: postprocessing time: 0.0282 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Uni

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 19.5615 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30117 rows (same as raw) in 1.4852 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 21.1131 sec.
Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Executor - Postprocessor: postprocessing time: 0.026 sec.
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univ

...

Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.1096 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Executor - Evaluator: anonymeter-singlingout-univariate at 1 trials evaluating time: 3.0354 sec.
{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.9300148011448004, 'Attack_Rate': 0.3967253428113813, 'Attack_Rate_err': 0.3967253428113813, 'Baseline_Rate': 0.3967253428113813, 'Baseline_Rate_err': 0.3967253428113813, 'Control_Rate': 0.3967253428113813, 'Control_Rate_err': 0.3967253428113813}
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0467 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.



Synthesizing: 100%|██████████| 2/2 [00:39<00:00, 19.54s/it]

Executor - Evaluator: anonymeter-singlingout-univariate at 2 trials evaluating time: 3.0829 sec.
{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.9300148011448004, 'Attack_Rate': 0.3967253428113813, 'Attack_Rate_err': 0.3967253428113813, 'Baseline_Rate': 0.3967253428113813, 'Baseline_Rate_err': 0.3967253428113813, 'Control_Rate': 0.3967253428113813, 'Control_Rate_err': 0.3967253428113813}
====== ====== ====== ====== ====== ======
Executor (run): Total execution time: 57.5893 sec.
====== ====== ====== ====== ====== ======


{'Risk': 0.0,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.9300148011448004,
 'Attack_Rate': 0.3967253428113813,
 'Attack_Rate_err': 0.3967253428113813,
 'Baseline_Rate': 0.3967253428113813,
 'Baseline_Rate_err': 0.3967253428113813,
 'Control_Rate': 0.3967253428113813,
 'Control_Rate_err': 0.3967253428113813}

In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment


para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
    ,'Evaluator_evaluating_method'    : 'anonymeter-inference'
    ,'Evaluator_anonymeter_n_attacks' : 1 # 2000
    ,'Evaluator_num_samples'          : 2
}

executor_single = PETs_Experiment.Executor(**para_Executor)
executor_single.run_single_process()
next(iter(
    executor_single.evaluator[('[Adt Income] adult.csv',
        '0.8',
        1,
        'drop-IQR-Label-Standard',
        'sdv-singletable-gaussiancoupula',
        'anonymeter-inference',
        1)].values())).Evaluator.evaluation

executor_single = PETs_Experiment.Executor(**para_Executor)
executor_single.run()
next(iter(
    executor_single.evaluator[('[Adt Income] adult.csv',
        '0.8',
        1,
        'drop-IQR-Label-Standard',
        'sdv-singletable-gaussiancoupula',
        'anonymeter-inference',
        1)].values())).Evaluator.evaluation

Executor - Loader: [Adt Income] adult.csv loading time: 10.949 sec.
Executor - Splitter: 0.8 splitting time: 0.1356 sec.
Preprocessor - Outlierist (IQR): Dropped  1069 rows on fnlwgt         . Kept [-63993.75, 419672.25] only.
Preprocessor - Outlierist (IQR): Dropped  3032 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   223 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped   236 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1734 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6092 in 36204 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column income          been labelized fr

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 16.8337 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30112 rows (same as raw) in 1.6529 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 23.7199 sec.
Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Executor - Postprocessor: postprocessing time: 0.0495 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (A

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 19.1928 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30136 rows (same as raw) in 1.7195 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 20.9434 sec.
Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Executor - Postprocessor: postprocessing time: 0.022 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (An

...

Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.001 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.2821 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.2329 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.077 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.0709 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evalua

Evaluator (Anonymeter): Evaluating  Inference spent 1.3364 sec.
Executor - Evaluator: anonymeter-inference at 1 trials evaluating time: 18.2062 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.001 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.7514 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.4574 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.001 sec.
Evaluator (Anonymeter): Evaluating  Inference.
Evaluator (Anonymeter): Evaluating  Inference spent 1.3427 sec.
Evaluator (Anonymeter - Inference): Now is Inference Evaluator
Evaluator (Anonymeter - Inference): Evaluator time: 0.0 sec.
Evaluator (Anonyme


Synthesizing: 100%|██████████| 2/2 [01:32<00:00, 46.22s/it]

Evaluator (Anonymeter): Evaluating  Inference spent 0.9761 sec.
Executor - Evaluator: anonymeter-inference at 2 trials evaluating time: 17.7892 sec.
====== ====== ====== ====== ====== ======
Executor (run): Total execution time: 107.3642 sec.
====== ====== ====== ====== ====== ======


{'Risk': 0.0022375332675201006,
 'Risk_CI_btm': 0.0,
 'Risk_CI_top': 0.046174667591818215,
 'Attack_Rate': 0.11495824409908227,
 'Attack_Rate_err': 0.027695443904980018,
 'Baseline_Rate': 0.07724796903662126,
 'Baseline_Rate_err': 0.023086784285609292,
 'Control_Rate': 0.11297349278000537,
 'Control_Rate_err': 0.027482077623402282}

In [19]:
import pandas as pd
for i in ['01','02','03']: # ,'04','05','06'
    eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-singlingout-univariate'
                                    ,data = {'ori'     : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
                                            ,'syn'     : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv")
                                            ,'control' : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                    }
                                    ,anonymeter_n_attacks = 500
                                    )
    eval.eval()
    print(eval.Evaluator.evaluation)

Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0549 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 203 failed queries out of 500. Check DEBUG messages for more details.


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.005411853750198381, 'Attack_Rate': 0.0038121702307761206, 'Attack_Rate_err': 0.00381217023077612, 'Baseline_Rate': 0.0038121702307761206, 'Baseline_Rate_err': 0.00381217023077612, 'Control_Rate': 0.0038121702307761206, 'Control_Rate_err': 0.00381217023077612}
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.062 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 189 failed queries out of 500. Check DEBUG messages for more details.


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.005411853750198381, 'Attack_Rate': 0.0038121702307761206, 'Attack_Rate_err': 0.00381217023077612, 'Baseline_Rate': 0.0038121702307761206, 'Baseline_Rate_err': 0.00381217023077612, 'Control_Rate': 0.0038121702307761206, 'Control_Rate_err': 0.00381217023077612}
Evaluator (Anonymeter - SinglingOut - Univariate): Now is SinglingOut - Univariate Evaluator
Evaluator (Anonymeter - SinglingOut - Univariate): Evaluator time: 0.0649 sec.
Evaluator (Anonymeter): Evaluating  SinglingOut - Univariate.


Found 210 failed queries out of 500. Check DEBUG messages for more details.


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.005411853750198381, 'Attack_Rate': 0.0038121702307761206, 'Attack_Rate_err': 0.00381217023077612, 'Baseline_Rate': 0.0038121702307761206, 'Baseline_Rate_err': 0.00381217023077612, 'Control_Rate': 0.0038121702307761206, 'Control_Rate_err': 0.00381217023077612}


In [4]:
def Result(__evaluator): 
    import numpy as np
    __dict_result    = {}
    __para_to_handle = [('Risk'              , ['risk()'    ,'value'                 ])
                        ,('Risk_CI_btm'       , ['risk()'    ,'ci[0]'                 ])
                        ,('Risk_CI_top'       , ['risk()'    ,'ci[1]'                 ])
                        ,('Attack_Rate'       , ['results()' ,'attack_rate'   ,'value'])
                        ,('Attack_Rate_err'   , ['results()' ,'attack_rate'   ,'error'])
                        ,('Baseline_Rate'     , ['results()' ,'baseline_rate' ,'value'])
                        ,('Baseline_Rate_err' , ['results()' ,'baseline_rate' ,'error'])
                        ,('Control_Rate'      , ['results()' ,'control_rate'  ,'value'])
                        ,('Control_Rate_err'  , ['results()' ,'control_rate'  ,'error'])
                        ]
    for __key ,__attrs in __para_to_handle:
        try:
            __attr_value = __evaluator
            for __attr in __attrs:
                if '()' in __attr:
                    __method_name = __attr.split('(')[0]
                    if hasattr(__attr_value ,__method_name):
                        __method = getattr(__attr_value ,__method_name)
                        if callable(__method):
                            __attr_value = __method()
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                elif '[' in __attr:
                    __attr_name = __attr.split('[')[0]
                    __index = int(__attr.split('[')[1].rstrip(']'))
                    if hasattr(__attr_value, __attr_name)\
                    and isinstance(getattr(__attr_value, __attr_name) ,(list, dict, tuple)):
                        try:
                            __attr_value = getattr(__attr_value  ,__attr_name)[__index]
                        except (IndexError, KeyError):
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __dict_result[__key] = np.nan
                        break
                else:
                    __attr_value = getattr(__attr_value, __attr)
            __dict_result[__key] = __attr_value
        except Exception as e:
            __dict_result[__key] = np.nan
    return __dict_result



import pandas as pd
for i in ['01','02','03']: # ,'04','05','06'
    from anonymeter.evaluators import SinglingOutEvaluator
    print(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
    evaluator = SinglingOutEvaluator(ori       = pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
                                    ,syn       = pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv")
                                    ,control   = pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                    ,n_attacks = 2000
                                    )
    try:
        evaluator.evaluate(mode='univariate')
        print(Result(evaluator))
    except RuntimeError as ex:
        print(f"Singling out evaluation failed with {ex}."
                "Please re-run this cell."
                "For more stable results increase `n_attacks`. Note that this will "
                "make the evaluation slower.")

PETsARD[20231224-085805]_Trial[01][Ori].csv


c:\Program Files\Python310\lib\site-packages\anonymeter\evaluators\singling_out_evaluator.py:97: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(values):
Found 813 failed queries out of 2000. Check DEBUG messages for more details.
c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215: UserWarning: Attack is as good or worse as baseline model. Estimated rates: attack = 0.0009585236406264672, baseline = 0.0009585236406264672. Analysis results cannot be trusted.
  self._sanity_check()


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.0013568577126237004, 'Attack_Rate': 0.0009585236406264672, 'Attack_Rate_err': 0.0009585236406264671, 'Baseline_Rate': 0.0009585236406264672, 'Baseline_Rate_err': 0.0009585236406264671, 'Control_Rate': 0.0009585236406264672, 'Control_Rate_err': 0.0009585236406264671}
PETsARD[20231224-085805]_Trial[02][Ori].csv


c:\Program Files\Python310\lib\site-packages\anonymeter\evaluators\singling_out_evaluator.py:97: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(values):
Found 772 failed queries out of 2000. Check DEBUG messages for more details.
c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215: UserWarning: Attack is as good or worse as baseline model. Estimated rates: attack = 0.0009585236406264672, baseline = 0.0009585236406264672. Analysis results cannot be trusted.
  self._sanity_check()


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.0013568577126237004, 'Attack_Rate': 0.0009585236406264672, 'Attack_Rate_err': 0.0009585236406264671, 'Baseline_Rate': 0.0009585236406264672, 'Baseline_Rate_err': 0.0009585236406264671, 'Control_Rate': 0.0009585236406264672, 'Control_Rate_err': 0.0009585236406264671}
PETsARD[20231224-085805]_Trial[03][Ori].csv


c:\Program Files\Python310\lib\site-packages\anonymeter\evaluators\singling_out_evaluator.py:97: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(values):
Found 789 failed queries out of 2000. Check DEBUG messages for more details.


{'Risk': 0.0, 'Risk_CI_btm': 0.0, 'Risk_CI_top': 0.0013568577126237004, 'Attack_Rate': 0.0009585236406264672, 'Attack_Rate_err': 0.0009585236406264671, 'Baseline_Rate': 0.0009585236406264672, 'Baseline_Rate_err': 0.0009585236406264671, 'Control_Rate': 0.0009585236406264672, 'Control_Rate_err': 0.0009585236406264671}


c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215: UserWarning: Attack is as good or worse as baseline model. Estimated rates: attack = 0.0009585236406264672, baseline = 0.0009585236406264672. Analysis results cannot be trusted.
  self._sanity_check()


In [ ]:
import pandas as pd
for i in ['01','02','03']: # ,'04','05','06'
    eval = PETs_Experiment.Evaluator(evaluating_method = 'anonymeter-singlingout-univariate'
                                    ,data = {'ori'     : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ori].csv")
                                            ,'syn'     : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}-1-1]Postproc.csv")
                                            ,'control' : pd.read_csv(f"PETsARD[20231224-085805]_Trial[{i}][Ctrl].csv")
                                    }
                                    ,anonymeter_n_attacks = 500
                                    )
    eval.eval()
    print(eval.Evaluator.evaluation)

In [18]:
import itertools
for type in ['][Ori]' ,'][Ctrl]' ,'-1-1]Postproc']:
    for combo in itertools.combinations(['01','02','03','04','05','06'], 2):
        df_a = pd.read_csv(f"PETsARD[20231224-085805]_Trial[{combo[0]}{type}.csv")
        df_b = pd.read_csv(f"PETsARD[20231224-085805]_Trial[{combo[1]}{type}.csv")
        if df_a.equals(df_b):
            print(type+': '+str(combo))
            print("They're same!!??")
        # else:
            # print(type+': '+str(combo))
            # print("They're different.")
print('done.')

][Ori]: ('01', '02')
They're different.
][Ori]: ('01', '03')
They're different.
][Ori]: ('01', '04')
They're different.
][Ori]: ('01', '05')
They're different.
][Ori]: ('01', '06')
They're different.
][Ori]: ('02', '03')
They're different.
][Ori]: ('02', '04')
They're different.
][Ori]: ('02', '05')
They're different.
][Ori]: ('02', '06')
They're different.
][Ori]: ('03', '04')
They're different.
][Ori]: ('03', '05')
They're different.
][Ori]: ('03', '06')
They're different.
][Ori]: ('04', '05')
They're different.
][Ori]: ('04', '06')
They're different.
][Ori]: ('05', '06')
They're different.
][Ctrl]: ('01', '02')
They're different.
][Ctrl]: ('01', '03')
They're different.
][Ctrl]: ('01', '04')
They're different.
][Ctrl]: ('01', '05')
They're different.
][Ctrl]: ('01', '06')
They're different.
][Ctrl]: ('02', '03')
They're different.
][Ctrl]: ('02', '04')
They're different.
][Ctrl]: ('02', '05')
They're different.
][Ctrl]: ('02', '06')
They're different.
][Ctrl]: ('03', '04')
They're d

In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'     : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 2 # 30
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 2 # 30
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor' : {
        'drop-stanard' : {'missing_method'  : 'drop'
                  ,'outlier_method'  : 'IQR'
                  # ,'outlier_columns' : {'ignore': ['hours-per-week']}
                  ,'encoding_method' : 'Label'
                  ,'scaling'         : True
                  ,'scaling_method'  : 'Standard'
                #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
        }
        ,'drop-minmax' : {'missing_method'  : 'drop'
                  ,'outlier_method'  : 'IQR'
                  # ,'outlier_columns' : {'ignore': ['hours-per-week']}
                  ,'encoding_method' : 'Label'
                  ,'scaling'         : True
                  ,'scaling_method'  : 'minmax'
                #   ,'scaling_columns' : {'focus' : 'fnlwgt'}
        }
    }
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
    #    ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
    #    ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
    ,'Evaluator' : {
        'SinglingOut' : {'method' : 'anonymeter-singlingout-univariate'
                         ,'num_samples' : 2 # 30
                         ,'anonymeter_n_attacks' : 2 # 2000
                         }
       ,'Linkability' : {'method' : 'anonymeter-linkability'
                         ,'num_samples' : 2 # 30
                         ,'anonymeter_n_attacks'   : 2 # 2000
                         ,'anonymeter_n_neighbors' : 10
                         ,'anonymeter_aux_cols'    : [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
                                                     ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
                                                     ]
                         }
       ,'Inference'   : {'method' : 'anonymeter-inference'
                         ,'num_samples' : 2 # 30
                         ,'anonymeter_n_attacks' : 2 # 2000
                         ,'anonymeter_secrets'   : True
                         }
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

Loading: 100%|██████████| 2/2 [00:19<00:00,  9.15s/it]















Synthesizing:   0%|          | 0/32 [00:00<?, ?it/s]


Synthesizer (SDV - SingleTable): Metafile loading time: 0.012 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.019 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0336 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.011 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.011 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.1526 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.1961 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.1976 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'mar'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarnin

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0288 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0669 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0843 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.2427 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarnin

Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 10.0239 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 10.3659 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 10.5791 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 11.1593 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 11.2124 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 103 rows (same as raw) in 1.4053 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 18.0615 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0141 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 12.6544 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 103 rows (same as raw) in 1.6235 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 19.2158 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.064 sec.
Synt

c:\Program Files\Python310\lib\site-packages\scipy\stats\_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 103 rows (same as raw) in 0.3844 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 9.9237 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0562 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 10.3872 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 103 rows (same as raw) in 0.7207 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 11.1638 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.1047 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.2173 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 103 rows (same as raw) in 0.3041 sec.
Executor - Synthesizer: GaussianCoupula synthesizing time: 14.5614 sec.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.008 sec.
Synthesizer (SDV - Si